In [19]:
import codecs
import re
import math
import nltk
import collections
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
#Токенизация текста
def tokenization(text):
    
    doc = re.split(r' *[\;\:\—\<\>\\\"\.\,\!\?\&\=\+\(\)\{\}\[\]\r\n\t\« \»\“\”\„\/\d ]', text) 
    tokens = []
    for token in doc: 
        if token != '':
            tokens.append(token.lower())        
    return tokens

In [3]:
#токенизируем каждый текст в корпусе
def tokenization_corpus(corpus):
    corpus_tokins = []
    for text in corpus:
        corpus_tokins.append(tokenization(text))
    return corpus_tokins

In [4]:
# получаем все токены в корпусе
def all_corpus_tokens(corpus_tokens):
    all_tokens =[]
    for text in corpus_tokens:
        for word in text:
            all_tokens.append(word)
    return all_tokens

In [5]:
#стемминг и лемматизация для одного текста
def stem_lem(tokens):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    return [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens]
    #return [stemmer.stem(word) for word in tokens]
    #return [lemmatizer.lemmatize(word) for word in tokens]

In [6]:
#стемминг и лемматизация для корпуса текстов
def stem_lem_corpus(corpus):
    u_corp = []
    for tokens in corpus:
        u_corp.append(stem_lem(tokens))
    return u_corp

In [101]:
#считаем P(wi)
def  Pw(tokens):
    tf_countter = collections.Counter(tokens)
    for i in tf_countter:
        tf_countter[i] = tf_countter[i]/len(tokens)
    return tf_countter

In [13]:
pathCV = './../3. TF-IDF/example1/' #Computer vision articles
texts = []
#получаем каждый документ и создаем корпус
for i in range(3):
    fileObj = codecs.open( pathCV + str(i)+'.txt', "r", "utf_8_sig" )
    text = fileObj.read() # или читайте по строке
    texts.append(text)
    fileObj.close()


In [14]:
tokens_corp = tokenization_corpus(texts)
all_tokens = all_corpus_tokens(tokens_corp)
    
    
#Лемматизация и стемминг
st_lm_all_tokkens = stem_lem(all_tokens)
st_lm_tokens_corp = stem_lem_corpus(tokens_corp)

In [161]:
def p_wi_wj(corpus,tokens,w_size):
    matrix_ww = np.zeros((len(tokens),len(tokens)))
    size_corpus = len(corpus)
    for i in range(size_corpus):
        left_border = 0 if i-w_size < 0 else i-w_size
        right_border = size_corpus if i+w_size > size_corpus else i+w_size
        #index_i_matrix = tokens.index(corpus[i]) 
        for j in np.arange(left_border,right_border):
            index_j_matrix = tokens.index(corpus[j])
            for k in np.arange(j+1,right_border):
                index_k_matrix = tokens.index(corpus[k])
                matrix_ww[index_k_matrix,index_j_matrix] += 1/size_corpus
                matrix_ww[index_j_matrix,index_k_matrix] += 1/size_corpus
                
    return matrix_ww

In [162]:
corpus_new = []
for text in st_lm_tokens_corp:
    corpus_new.extend(text)

In [163]:
unique = list(set(st_lm_all_tokkens))
unique.index('vision')

685

In [164]:
len(corpus_new)

9198

In [188]:
matrix_wi_wj = p_wi_wj(corpus_new,unique,2)

In [181]:
matrix_wi_wj.shape

(1474, 1474)

In [182]:
counter_wi = Pw(st_lm_all_tokkens)
len(st_lm_all_tokkens)

9198

In [196]:
def ppmi(matrix_wi_wj,counter_wi,unique):
    size = len(unique)
    matrix_ppmi = np.zeros((size,size))
    for i in range(size):
        word_i = unique[i]
        for j in range(size):
            word_j = unique[j]
            if matrix_wi_wj[i,j] == 0:
                matrix_ppmi[i,j] = 0
            else:
                matrix_ppmi[i,j] = np.log(matrix_wi_wj[i,j]/(counter_wi[word_i]*counter_wi[word_j]))#PMI=P(Wi,Wj)/(P(Wi)*P(Wj))
    return matrix_ppmi     
            

In [197]:
matrix_ppmi = ppmi(matrix_wi_wj,counter_wi,unique)

In [185]:
matrix_ppmi

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
1204,685

In [195]:
for it in matrix_ppmi:
    for ti in it:
        if ti<0:
            print(ti)

-0.31065378406395444
-0.7147742252205117
-0.21389762717099223
-0.21389762717099223
-0.19195870129835865
-0.31065378406395444
-0.7147742252205117
-0.3158294904223601
-0.19195870129835865


In [198]:
for it in matrix_ppmi:
    for ti in it:
        if ti<0:
            print(ti)

-0.21532879455420809
-0.49544373894851707
-0.1482625372020356
-0.1482625372020356
-0.13305563258890601
-0.21532879455420809
-0.49544373894851707
-0.21891632082394313
-0.13305563258890601


In [174]:
matrix_wi_wj[1204,685]

0.09502065666449228

In [175]:
874/9198

0.09502065666449228

In [201]:
from sklearn.metrics.pairwise import cosine_similarity

In [214]:
cs_m = cosine_similarity(matrix_ppmi)

In [221]:
maxn = 0
x = 0
y = 0
for i,simpl in enumerate(cs_m):
    for j,ti in enumerate(simpl):
        if ti>maxn and ti<1-0.001:
            x = i
            y = j
            maxn = ti
            
            

In [222]:
x,y,maxn

(883, 974, 0.8340186277625213)

In [224]:
unique[882],unique[973]

('surveil', 'realis')